# Using ctypes, a foreign function library for Python.
https://docs.python.org/3.5/library/ctypes.html

## 1. execute a function from an existing library

In [ ]:
! man 2 time

In [6]:
from ctypes import CDLL

libc = CDLL("libc.so.6") 
print(libc.time())

1496740758


## 2. execute a function from your own (C) library

### 2.1 by passing parameter by value

#### Header file sum.h

In [ ]:
#ifndef __REDUCTION_H__
#define  __REDUCTION_H__
    float sum(float a, float b);
#endif

#### C file sum.c

In [ ]:
float sum(float a, float b){
    float result = a + b;
    return result;
}

#### commands to build the shared library

In [ ]:
! rm -rf libsum.so sum.o
! gcc -fPIC -Wall -c sum.c -I. -o sum.o
! gcc -shared  -o libsum.so sum.o
! ls

In [ ]:
from ctypes import CDLL
from ctypes import c_float

lib = CDLL("libsum.so")
lib.sum.restype = c_float
lib.sum.argtypes = [c_float, c_float]

a = 3.43
b = 2.65
result =  lib.sum(a, b)
print('{} +  {} = {:.2f}'.format(a, b, result))

### 2.2 by passing parameter by address (~by reference)

#### header array_sum.h

In [ ]:
#ifndef __REDUCTION_H__
#define  __REDUCTION_H__

float array_sum(float* tab, unsigned int size);

#endif

#### C file array_sum.c

In [ ]:
/// Sum of array elements.
/**	@param tab  : float pointer ,    array to sum
 * 	@param size : unsigned int  ,    array size
 * 	@return     : float         ,    sum of array elements
*/
float array_sum(float* tab, unsigned int size){

    float result = 0.;
        for ( unsigned int i =0; i < size; ++i){
            result += tab[i];
        }
        return result;
}


In [ ]:
! gcc -fPIC -Wall -c array_sum.c -I. -o array_sum.o
! gcc -shared  -o libarraysum.so array_sum.o
! ls

In [7]:
from ctypes import CDLL
from ctypes import c_float
from ctypes import c_uint
from ctypes import POINTER

lib = CDLL("libarraysum.so")
lib.array_sum.restype = c_float
lib.array_sum.argtypes = [POINTER(c_float),c_uint]

l = [1,2,3,4,5]
arr = (c_float * len(l))(*l)
result = lib.array_sum(arr, len(l))
print('sum {} = {:.2f}'.format(l, result))

sum [1, 2, 3, 4, 5] = 15.00


Note : a star before a list will unpack it
`l = [1,2,3,4,5]`
`arr = (c_float * len(l))(*l)`
is equivalent to
`arr = (c_float * len(l))(1,2,3,4,5)`